In [1]:
from scapy.all import *
load_layer("ssl")
load_layer("tls")
import pprint
import json, time
from os import listdir
from os.path import isfile, join
import pandas as pd
mypath = "/home/bx1/NUS/DoH Analysis/data2"  # path where the data is stored

In [2]:
# get urls from alexadomain.json 
rank_from = 1
rank_to = 2

with open("alexadomain.json") as jsf:
        data = json.load(jsf)

for i in range(rank_from-1,rank_to):
    print(data[i]["DataUrl"])

google.com
youtube.com


In [3]:
data[1687]

{'DataUrl': '24smi.info',
 'Country': {'Rank': '1687',
  'Reach': {'PerMillion': '505'},
  'PageViews': {'PerMillion': '24.7', 'PerUser': '1.77'}},
 'Global': {'Rank': '1687'}}

In [4]:
# parse all urls.json files and store in <dns_res_stat>
data_dir = mypath + "/urls/"

onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]

dns_res_stat = []
for i in range(1,len(onlyfiles)+1):
    with open(data_dir + "url" + str(i) + ".json"  ,"r") as f:
        json_obj =  json.load(f)
        dns_res_stat.append(json_obj)
        
        
print(len(dns_res_stat))
dns_res_stat[0]

4000


{'ocsp.pki.goog': ['74.125.24.94', '2404:6800:4003:c04::5e'],
 'snippets.cdn.mozilla.net': ['13.224.253.29',
  '13.224.253.122',
  '13.224.253.72',
  '13.224.253.12'],
 'ocsp.digicert.com': ['117.18.237.29'],
 'tracking-protection.cdn.mozilla.net': ['13.224.253.120',
  '13.224.253.121',
  '13.224.253.76',
  '13.224.253.24'],
 'ogs.google.com': ['74.125.24.102',
  '74.125.24.101',
  '74.125.24.138',
  '74.125.24.113',
  '74.125.24.100',
  '74.125.24.139',
  '2404:6800:4003:c03::66'],
 'www.google.com.sg': ['172.217.194.94', '2404:6800:4003:c02::5e'],
 'apis.google.com': ['2404:6800:4003:80a::200e',
  '74.125.24.139',
  '74.125.24.100',
  '74.125.24.102',
  '74.125.24.101',
  '74.125.24.113',
  '74.125.24.138'],
 'www.gstatic.com': ['2404:6800:4003:c02::5e', '74.125.68.94'],
 'adservice.google.com': ['2404:6800:4003:c02::9c',
  '74.125.24.155',
  '74.125.24.157',
  '74.125.24.154',
  '74.125.24.156'],
 'www.google.com': ['172.217.194.147',
  '172.217.194.106',
  '172.217.194.103',
  '172

In [5]:
# function to get the time taken by the data collecor script (uses <query-time>)
def get_time_interval(dns_res_stat):
    st = dns_res_stat[0]["query-stime"]
    end = dns_res_stat[len(dns_res_stat)-1]["query-stime"]
    
    print("start-time: " + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(st)))
    print("end-time: " + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end)))
    print("Took " + str(end-st) + " seconds for " + str(len(dns_res_stat)) + " websites" )

In [6]:
# function to get the failed queries - due to URL format issue (https://www.) or network issue
def get_failed_queries(dns_res_stat):
    failed_query = []
    
    url_skip_stat = []
    for i in range(0,len(dns_res_stat)):
        if dns_res_stat[i]["query-status"] == 0:
            #pint(str(dns_res_stat[i]["query-url"]) + " : " + str(dns_res_stat[i]["query-stime"]))
            #print("----->" + str(dns_res_stat[i]))
            failed_query.append(i)
    return failed_query 

In [7]:
#statistics from the DNS resolutions from firefox 

get_time_interval(dns_res_stat)   
failed_query = get_failed_queries(dns_res_stat) #failed_query have the ranks of the URLS
len(failed_query) 

start-time: 2019-10-21 16:17:41
end-time: 2019-10-24 15:06:21
Took 254919.99915790558 seconds for 4000 websites


963

In [8]:
#individual websites which is failed 
dns_res_stat[failed_query[3]]["query-url"]

'http://www.yahoo.co.jp'

In [9]:
# parse pcaps and store in <pcap>
data_dir = mypath + "/traces/" 

query_no = 0
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]
onlyfiles = sorted(onlyfiles)

pcap = rdpcap(data_dir+onlyfiles[query_no])
tls_pcap = pcap[TLS]
len(pcap),len(tls_pcap)

(1046, 472)

In [10]:
pcap[0]

<Ether  dst=02:42:66:c2:79:b2 src=02:42:ac:11:00:02 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=60 id=17227 flags=DF frag=0 ttl=64 proto=tcp chksum=0x3b4e src=172.17.0.2 dst=8.8.8.8 |<TCP  sport=55136 dport=https seq=1045311707 ack=0 dataofs=10 reserved=0 flags=S window=64240 chksum=0xbc51 urgptr=0 options=[('MSS', 1460), ('SAckOK', b''), ('Timestamp', (2515866414, 0)), ('NOP', None), ('WScale', 7)] |>>>

In [11]:
# rough work to identify how to parse the fields and get correspoding values
temp = tls_pcap[0]
layert = type(temp[TCP].payload)
p = temp[layert].fields['msg'][0]
p.msgtype,p.ext[0].servernames[0].servername

(1, b'dns.google')

In [12]:
# get all the fields in field-value pairs from IP,TCP and TLS layers
temp_pcap = pcap
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
tls_fields = [field.name for field in TLS().fields_desc]
print(ip_fields,tcp_fields,tls_fields)

['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto', 'chksum', 'src', 'dst', 'options'] ['sport', 'dport', 'seq', 'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr', 'options'] ['type', 'version', 'len', 'iv', 'msg', 'mac', 'pad', 'padlen']


In [ ]:


########### Rough work BEGINs to get the fields and converting it into a DataFrame ###########



In [13]:
# rough work - IP fields into a Dataframe

df_fields = ip_fields + ['time'] #+ tcp_fields  #+ ['payload']
df = pd.DataFrame(columns=df_fields)

for pkt in temp_pcap[IP]:

    values = []    
    
    # IP layer fields
    for fd in ip_fields:
        
        if fd == 'options':
            values.append(len(pkt[IP].fields[fd]))
        else:
            values.append(pkt[IP].fields[fd])
    
    values.append(pkt.time)

    
    df_append = pd.DataFrame([values], columns=df_fields)
    df = pd.concat([df, df_append], axis=0)

df = df.reset_index()
df = df.drop(columns="index")
df[69:122]

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,src,dst,options,time
69,4,5,0,91,31851,DF,0,64,6,527,172.17.0.2,8.8.8.8,0,1571654860.349295
70,4,5,0,52,51500,,0,104,6,52596,8.8.8.8,172.17.0.2,0,1571654860.355563
71,4,5,0,60,30460,DF,0,64,6,39806,172.17.0.2,52.215.71.87,0,1571654860.409490
72,4,5,0,60,0,DF,0,227,6,28538,52.215.71.87,172.17.0.2,0,1571654860.575829
73,4,5,0,52,30461,DF,0,64,6,39813,172.17.0.2,52.215.71.87,0,1571654860.575851
74,4,5,0,569,30462,DF,0,64,6,39295,172.17.0.2,52.215.71.87,0,1571654860.576816
75,4,5,0,113,31852,DF,0,64,6,504,172.17.0.2,8.8.8.8,0,1571654860.601469
76,4,5,0,147,31853,DF,0,64,6,469,172.17.0.2,8.8.8.8,0,1571654860.601517
77,4,5,0,109,31854,DF,0,64,6,506,172.17.0.2,8.8.8.8,0,1571654860.601595
78,4,5,0,147,31855,DF,0,64,6,467,172.17.0.2,8.8.8.8,0,1571654860.601616


In [14]:
df.tos.value_counts()

0    1046
Name: tos, dtype: int64

In [15]:
df[df.id == 41572]

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,src,dst,options,time


In [25]:
# rough work - TCP fields into a Dataframe
tcp_fd = ['sport', 'dport', 'flags'] 

df_fields = tcp_fields  #+ ['payload']        
                

df = pd.DataFrame(columns=df_fields)

for pkt in temp_pcap[IP]:

    values = []    
    
    #tcp layer - getting type
    layer_type = type(pkt[IP].payload)
    for fd in tcp_fields:
        try:
            if fd == 'options':
                values.append(len(pkt[layer_type].fields[fd][0]))
            else:
                values.append(pkt[layer_type].fields[fd])
        except:
            values.append(None)
    
    df_append = pd.DataFrame([values], columns=df_fields)
    df = pd.concat([df, df_append], axis=0)

df = df.reset_index()
df = df.drop(columns="index")
df.head()

,sport,dport,seq,ack,dataofs,reserved,flags,window,chksum,urgptr,options
0,55136,443,1045311707,0,10,0,S,64240,48209,0,2
1,443,55136,2995151223,1045311708,10,0,SA,62392,56464,0,2
2,55136,443,1045311708,2995151224,8,0,A,502,48201,0,2
3,55136,443,1045311708,2995151224,8,0,PA,502,48718,0,2
4,443,55136,2995151224,1045312225,8,0,A,248,64449,0,2


In [ ]:
df.flags.value_counts()

In [17]:
# rough work - TLS fields into a Dataframe

tls_fields = ['type', 'version', 'len', 'iv', 'mac', 'pad', 'padlen', 'sni']
df_fields = tls_fields  #+ ['payload']

df = pd.DataFrame(columns=df_fields)

for pkt in temp_pcap[IP]:

    values = []    
    
    if type(pkt[TCP].payload) != scapy.packet.NoPayload: 
        #TLS layer - getting type is to just make sure
        layer_type = type(pkt[TCP].payload)
        for fd in tls_fields:
            try:
                if fd == 'sni':
                    if pkt[layer_type].fields['msg'] and pkt[layer_type].fields['msg'][0].msgtype == 1:
                        values.append(pkt[layer_type].fields['msg'][0].ext[0].servernames[0].servername)
                    else:
                        values.append(0)
                else:
                    values.append(pkt[layer_type].fields[fd])
            except:
                values.append(None)
    else:
        for fd in tls_fields:
            values.append(None)
    
#     values.append(len(pkt[layer_type].payload))
#     #field_values.append(packet[layer_type].payload.original)
#     #field_values.append(binascii.hexlify(packet[layer_type].payload.original))

    
    df_append = pd.DataFrame([values], columns=df_fields)
    df = pd.concat([df, df_append], axis=0)

df = df.reset_index()
df = df.drop(columns="index")
df.head()

,type,version,len,iv,mac,pad,padlen,sni
0,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None
3,22,769,512,b'',None,None,None,b'dns.google'
4,None,None,None,None,None,None,None,None


In [ ]:



####### Rough work ENDs ##########




In [18]:
# Shortlisting needed fields and creatign a Dataframe
ip_fd = ['src', 'dst', 'len', 'ttl'] 
tcp_fd = ['sport', 'dport', 'flags']  
tls_fd = ['type', 'version', 'sni']

df_fields = ip_fd + tcp_fd + tls_fd + ['time'] + ['query-url']

df = pd.DataFrame(columns=df_fields)

for pkt in temp_pcap[IP]:

    values = []    
    
    # IP layer fields
    for fd in ip_fd:
        
        if fd == 'options':
            values.append(len(pkt[IP].fields[fd]))
        else:
            values.append(pkt[IP].fields[fd])


    # TCP layer - getting type
    layer_type = type(pkt[IP].payload)
    for fd in tcp_fd:
        try:
            if fd == 'options':
                values.append(len(pkt[layer_type].fields[fd][0]))
            else:
                values.append(pkt[layer_type].fields[fd])
        except:
            values.append(None)
            
    
# TYPE:23 - application data
#     22 - handshake proto
#     21 - alert protocol
#     20 - change cipher spec
                   
    # TLS fields
    if type(pkt[TCP].payload) != scapy.packet.NoPayload: 
        layer_type = type(pkt[TCP].payload)
        for fd in tls_fd:
            try:
                if fd == 'sni':
                    if pkt[layer_type].fields['msg'] and pkt[layer_type].fields['msg'][0].msgtype == 1:
                        values.append(pkt[layer_type].fields['msg'][0].ext[0].servernames[0].servername)
                    else:
                        values.append(0)
                else:
                    values.append(pkt[layer_type].fields[fd])
            except:
                values.append(0)
    else:
        for fd in tls_fd:
            values.append(None)
    
    # Adding time and queried URL
    values.append(pkt.time)
    values.append(dns_res_stat[query_no]['query-url'])
    
#     values.append(len(pkt[layer_type].payload))
#     #field_values.append(packet[layer_type].payload.original)
#     #field_values.append(binascii.hexlify(packet[layer_type].payload.original))

    
    df_append = pd.DataFrame([values], columns=df_fields)
    df = pd.concat([df, df_append], axis=0)
    
df = df.reset_index()
df = df.drop(columns="index")
#df.columns = ['src', 'dst', 'iplen', 'ttl', 'sport', 'dport', 'flags', 'options', 'ack', 'dataofs', 'type', 'version', 'tlslen', 'sni', 'time', 'query-url']
df.head()

,src,dst,len,ttl,sport,dport,flags,type,version,sni,time,query-url
0,172.17.0.2,8.8.8.8,60,64,55136,443,S,None,None,None,1571654858.891679,http://www.google.com
1,8.8.8.8,172.17.0.2,60,104,443,55136,SA,None,None,None,1571654858.894632,http://www.google.com
2,172.17.0.2,8.8.8.8,52,64,55136,443,A,None,None,None,1571654858.894650,http://www.google.com
3,172.17.0.2,8.8.8.8,569,64,55136,443,PA,22,769,b'dns.google',1571654858.895607,http://www.google.com
4,8.8.8.8,172.17.0.2,52,104,443,55136,A,None,None,None,1571654858.898672,http://www.google.com


In [19]:
# get the packets with the SNIs
df[(df.sni != 0) & df.sni.notnull()]

,src,dst,len,ttl,sport,dport,flags,type,version,sni,time,query-url
3,172.17.0.2,8.8.8.8,569,64,55136,443,PA,22,769,b'dns.google',1571654858.895607,http://www.google.com
35,172.17.0.2,8.8.8.8,678,64,55166,443,PA,22,769,b'dns.google',1571654860.321700,http://www.google.com
74,172.17.0.2,52.215.71.87,569,64,38216,443,PA,22,769,b'location.services.mozilla.com',1571654860.576816,http://www.google.com
118,172.17.0.2,8.8.8.8,678,64,55182,443,PA,22,769,b'dns.google',1571654860.753560,http://www.google.com
164,172.17.0.2,54.149.19.17,569,64,42432,443,PA,22,769,b'shavar.services.mozilla.com',1571654860.847866,http://www.google.com
183,172.17.0.2,8.8.8.8,569,64,55188,443,PA,22,769,b'dns.google',1571654860.972715,http://www.google.com
211,172.17.0.2,8.8.8.8,569,64,55190,443,PA,22,769,b'dns.google',1571654862.471539,http://www.google.com
225,172.17.0.2,8.8.8.8,569,64,55192,443,PA,22,769,b'dns.google',1571654862.487472,http://www.google.com
309,172.17.0.2,172.217.194.147,569,64,52414,443,PA,22,769,b'www.google.com',1571654862.543603,http://www.google.com
342,172.17.0.2,13.224.253.29,569,64,38736,443,PA,22,769,b'snippets.cdn.mozilla.net',1571654862.597467,http://www.google.com


In [20]:
#get first packet after given time
for pkt in pcap:
    if pkt.time >= 1569299628.325551:
        pkt.show()
        break;

###[ Ethernet ]### 
  dst       = 02:42:66:c2:79:b2
  src       = 02:42:ac:11:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 17227
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x3b4e
     src       = 172.17.0.2
     dst       = 8.8.8.8
     \options   \
###[ TCP ]### 
        sport     = 55136
        dport     = https
        seq       = 1045311707
        ack       = 0
        dataofs   = 10
        reserved  = 0
        flags     = S
        window    = 64240
        chksum    = 0xbc51
        urgptr    = 0
        options   = [('MSS', 1460), ('SAckOK', b''), ('Timestamp', (2515866414, 0)), ('NOP', None), ('WScale', 7)]



In [ ]:



####### Rough work BEGINs for getting more values from TLS field ##########




In [22]:
tls_pcap[43].show()

###[ Ethernet ]### 
  dst       = 02:42:66:c2:79:b2
  src       = 02:42:ac:11:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 147
     id        = 31855
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x1d3
     src       = 172.17.0.2
     dst       = 8.8.8.8
     \options   \
###[ TCP ]### 
        sport     = 55166
        dport     = https
        seq       = 691474357
        ack       = 3576900200
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 501
        chksum    = 0xbca8
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (2515868124, 507616863))]
###[ TLS ]### 
           type      = application_data
           version   = TLS 1.2
           len       = 90    [deciphered_len= 90]
           iv        = b''
           \msg       \
            |###[ TLS Application Data ]### 
            |  

In [ ]:
len(tls_pcap)

In [23]:
# trial and error method for checking and handling type of TLS packet(hackshake/app-data..) 
# and their respective numbers which are assigned to them in the packet (eg: client-hello:1 and server-key-exch:12)
# caught[] will contain which are not handled and hence can be added to if statements. 
# This code will be further moved to portion where TLS fields are handles and made into Dataframe
caught = []
for k in range(0,len(tls_pcap)):
    #record_stat = [0,0,0,0]
    temp = tls_pcap[k]
    layert = type(temp[TCP].payload)
    temp = temp[layert]
    p = temp.fields['msg'][0]
    # temp.fields['type'],p.msgtype,p.ext[0].servernames[0].servername #client hello
    #print(temp.fields['type'],p.msgtype)


    #print("Length:",len(temp.layers()))
    for i in temp.iterpayloads():
        fld_val = i.fields

        try:
            if fld_val['type']:
                if fld_val['type'] == 22:                      #handshake

                    #print(fld_val['type'])
                    try:
                        inter_fld_val = fld_val['msg'][0].fields
                    except:
                        #print("Handshake raw")
                        f=1

                    
                    if inter_fld_val['msgtype'] == 0:          
                        #print("0:Hello request")
                        f=1

                    elif inter_fld_val['msgtype'] == 1:            #client hello
                        #print("1:Client hello")
                        f=1

                    elif inter_fld_val['msgtype'] == 2:          #server hello
                        #print("2:Server hello")
                        f=1
                        
                    elif inter_fld_val['msgtype'] == 4:          
                        #print("4:Session Ticket")
                        f=1

                    elif inter_fld_val['msgtype'] == 11:         
                        #print("11:Certificate")
                        f=1

                    elif inter_fld_val['msgtype'] == 12:          
                        #print("Server Key exchange")
                        f=1

                    elif inter_fld_val['msgtype'] == 14:          
                        #print("Server Hello done")
                        f=1

                    elif inter_fld_val['msgtype'] == 16:          
                        #print("Client Key exchange")
                        f=1
                    
                    elif inter_fld_val['msgtype'] == 22:          
                        #print("22:Certificate Status")
                        f=1
                    
            

                    else:
                        #print("Caught!!!!!!!! from handshake: ",inter_fld_val['msgtype'])
                        caught.append(k)

                elif fld_val['type'] == 20:                    #change cipher spec
                    #print(fld_val['type'], "Change cipher spec")
                    f=1
                    
                elif fld_val['type'] == 21:                    # alert - close_notify
                    #print(fld_val['type'], "Alert")
                    f=1
                
                elif fld_val['type'] == 23:                    #application data
                    #print(fld_val['type'], "Application data")  
                    f=1
                else:
                    print("Caught!!!!!!!!:", fld_val['type'])
                    caught.append(k)
        except:
            #print("Raw data")
            f=1
           
    #print(str(k)+" packet finished")
    #print("-----------------")
print(caught)

[]


In [ ]:
# same as the code above but handles all files in the folder 
data_dir = mypath + "/traces/" 

files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]

files = sorted(files)

for query_no in range(0,len(files)):

    pcap = rdpcap(data_dir+files[query_no])
    tls_pcap = pcap[TLS]

    caught = []
    for k in range(0,len(tls_pcap)):
        record_stat = [0,0,0,0]
        temp = tls_pcap[k]
        layert = type(temp[TCP].payload)
        temp = temp[layert]
        p = temp.fields['msg'][0]
        # temp.fields['type'],p.msgtype,p.ext[0].servernames[0].servername #client hello
        #print(temp.fields['type'],p.msgtype)


        #print("Length:",len(temp.layers()))
        for i in temp.iterpayloads():
            fld_val = i.fields

            try:
                if fld_val['type']:
                    if fld_val['type'] == 22:                      #handshake

                        #print(fld_val['type'])
                        try:
                            inter_fld_val = fld_val['msg'][0].fields
                        except:
                            #print("Handshake raw")
                            f=1


                        if inter_fld_val['msgtype'] == 0:          
                            #print("0:Hello request")
                            f=1

                        elif inter_fld_val['msgtype'] == 1:            #client hello
                            #print("1:Client hello")
                            f=1

                        elif inter_fld_val['msgtype'] == 2:          #server hello
                            #print("2:Server hello")
                            f=1

                        elif inter_fld_val['msgtype'] == 4:          #server hello
                            #print("4:Session Ticket")
                            f=1

                        elif inter_fld_val['msgtype'] == 11:         
                            #print("11:Certificate")
                            f=1

                        elif inter_fld_val['msgtype'] == 12:          
                            #print("Server Key exchange")
                            f=1

                        elif inter_fld_val['msgtype'] == 14:          
                            #print("Server Hello done")
                            f=1

                        elif inter_fld_val['msgtype'] == 16:          
                            #print("Client Key exchange")
                            f=1

                        elif inter_fld_val['msgtype'] == 22:          
                            #print("22:Certificate Status")
                            f=1



                        else:
                            print("Caught!!!!!!!! from handshake: ",inter_fld_val['msgtype'])
                            caught.append(k)

                    elif fld_val['type'] == 20:                    #change cipher spec
                        #print(fld_val['type'], "Change cipher spec")
                        f=1

                    elif fld_val['type'] == 21:                    # alert - close_notify
                        #print(fld_val['type'], "Alert")
                        f=1

                    elif fld_val['type'] == 23:                    #application data
                        #print(fld_val['type'], "Application data")  
                        f=1
                    else:
                        print("Caught!!!!!!!!:", fld_val['type'])
                        caught.append(k)
            except:
                #print("Raw data")
                f=1

        #print(str(k)+" packet finished")
        #print("-----------------")
    print(query_no,":",caught)

In [ ]:
temp.layers(),type(temp[0])

In [ ]:
temp[0].show()

In [ ]:
type(temp[0].payload.payload), type(temp[0])

In [ ]:
temp[0].fields['msg'][0].fields

In [ ]:
ethernet_frame = tls_pcap[1]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload

print(ethernet_frame.summary())
print(ip_packet.summary())
print(segment.summary())
print(data.summary())

In [ ]:
tls = segment.payload
tls.show()

In [ ]:
tls.msg[0].gmt_unix_time

In [ ]:
#msgtype = 2 is for server_hello
tls.msg[0].msgtype

In [ ]:
for tls_pkt in pcap[TLS]:
    tls = tls_pkt.payload.payload.payload
    # "handshake" (not application data) and "client_hello"
    if tls.type == 22 and tls.msg[0].msgtype == 1:
        print(tls.msg[0].ext[0].servernames[0].servername.decode('UTF-8'))